##### Import statements

In [1]:
import os
os.environ[ "TF_CPP_MIN_LOG_LEVEL"] = "3"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.logging.set_verbosity( tf.compat.v1.logging.ERROR)
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
dataset = pd.read_csv( "airline-passengers.csv", parse_dates=[ "Month"], engine="python", index_col="Month")

In [ ]:
dataset.head( 5)

In [ ]:
plt.figure()
plt.plot( dataset)
plt.show()

In [ ]:
data = dataset[ "Passengers"].values.reshape( -1, 1).astype( "float32")

In [ ]:
scaler = MinMaxScaler( feature_range=( 0, 1))
data = scaler.fit_transform( data)

In [ ]:
train_size = int( len( dataset) * 0.67)
test_size = len( dataset) - train_size
train, test = data[ 0:train_size, :], data[ train_size - 1:len( dataset), :]

print( f"shape of train set: { len( train)}")
print( f"shape of test set: { len( test)}")

In [ ]:
def create_dataset( dataset, lag=1):
	dataX, dataY = [], []

	for i in range( len( dataset) - lag - 1):
		a = dataset[ i:( i + lag), 0]
		dataX.append( a)
		dataY.append( dataset[ i + lag, 0])
	return np.array( dataX), np.array( dataY)

In [ ]:
lag = 1

train_X, train_y = create_dataset( train, lag)
test_X, test_y = create_dataset( test, lag)

print( f"train_X shape: { train_X.shape}, train_y shape: { train_y.shape}")
print( f"test_X shape: { test_X.shape}, test_y shape: { test_y.shape}")

In [ ]:
train_X = np.reshape( train_X, ( train_X.shape[ 0], lag, train_X.shape[ 1]))
test_X = np.reshape( test_X, ( test_X.shape[ 0], lag, test_X.shape[ 1]))

print( f"train_X shape: { train_X.shape}, train_y shape: { train_y.shape}")
print( f"test_X shape: { test_X.shape}, test_y shape: { test_y.shape}")

In [ ]:
model = Sequential( [
	LSTM( 4, input_shape=( lag, 1)),
	Dense( 1)
])

model.compile(
	loss="mean_squared_error",
	optimizer="adam"
)

history = model.fit(
	train_X,
	train_y,
	epochs=100,
	batch_size=1,
	verbose=0
)

In [ ]:
def plot_loss( history):
	plt.figure()
	plt.plot( history.history[ "loss"])
	plt.xlabel( "Epochs")
	plt.title( "Training Loss")
	plt.show()

In [ ]:
plot_loss( history)

In [ ]:
def evaluate( model, lag, data, scaler, train_X, train_y, test_X, test_y):
	train_pred = model.predict( train_X, verbose=0)
	test_pred = model.predict( test_X, verbose=0)

	train_pred = scaler.inverse_transform( train_pred)
	train_true = scaler.inverse_transform( [ train_y])

	test_pred = scaler.inverse_transform( test_pred)
	test_true = scaler.inverse_transform( [ test_y])

	train_score = np.sqrt( mean_squared_error( train_true[ 0], train_pred[ :, 0]))
	print( f"Train Score: { train_score:.2f} RMSE")
	
	test_score = np.sqrt( mean_squared_error( test_true[ 0], test_pred[ :, 0]))
	print( f"Test Score: { test_score:.2f} RMSE")

	train_predict_plot = np.empty_like( data)
	train_predict_plot[ :, :] = np.nan
	train_predict_plot[ lag:len( train_pred) + lag, :] = train_pred

	test_predict_plot = np.empty_like( data)
	test_predict_plot[ :, :] = np.nan
	test_predict_plot[ len( train_pred) + lag * 2 + 1:len( data) - 1, :] = test_pred

	plt.figure( dpi=400)
	plt.plot( scaler.inverse_transform( data))
	plt.plot( train_predict_plot)
	plt.plot( test_predict_plot)
	plt.show()

In [ ]:
evaluate( model, lag, data, scaler, train_X, train_y, test_X, test_y)